In [122]:
__file__ = "__init__.py"

In [123]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch
import sys
import os
from pathlib import Path

project_root = Path(__file__).resolve().parents[1]
sys.path.append(str(project_root))

In [124]:
paths = {
    "processed": os.path.abspath(f"{project_root}/data/storage/processed"),
    "qfragments": os.path.abspath(f"{project_root}/intents/qfragments.json"),
    "questions": os.path.abspath(f"{project_root}/intents/questions.csv"),
    "model": os.path.abspath(f"{project_root}/models/t5-small"),
    "results": os.path.abspath(f"{project_root}/training/results"),
}

In [129]:
model_path = paths["model"]
model = T5ForConditionalGeneration.from_pretrained(model_path)
tokenizer = T5Tokenizer.from_pretrained(model_path)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def generate_output(input_text, model, tokenizer, device):
    # Mã hóa văn bản đầu vào và chuyển sang thiết bị phù hợp
    inputs = tokenizer(
        input_text, return_tensors="pt", max_length=512, truncation=True
    ).to(device)
    
    # Gọi model.generate với các tham số kiểm soát đầu ra
    outputs = model.generate(
        **inputs,
        max_length=128,
        num_beams=5,              # sử dụng beam search với 5 beams
        no_repeat_ngram_size=2,   # không lặp lại bất kỳ 2-gram nào
        temperature=0.7,          # điều chỉnh mức độ ngẫu nhiên
        top_k=50,                 # chỉ cân nhắc top 50 token có xác suất cao nhất
        top_p=0.9,                # chọn token sao cho tổng xác suất đạt 0.9
        repetition_penalty=1.2,   # áp dụng hình phạt cho token lặp lại
        early_stopping=True       # dừng sớm nếu mọi beam đều kết thúc
    )
    
    # Giải mã đầu ra thành văn bản
    decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return decoded_output

# Ví dụ sử dụng:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Giả sử bạn đã tải mô hình và tokenizer
# model = T5ForConditionalGeneration.from_pretrained("your_model_checkpoint")
# tokenizer = T5Tokenizer.from_pretrained("your_model_checkpoint")
model.to(device)

input_text = "Recommend me a laptop with RTX 4080, i9 14th."
result = generate_output(input_text, model, tokenizer, device)
print("Extracted components:", result)


Extracted components: Recomend me a laptop with RTX 4080, i9 14th.


In [126]:
# model_path = paths["model"]
# model = T5ForConditionalGeneration.from_pretrained(model_path)
# tokenizer = T5Tokenizer.from_pretrained(model_path)

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device)

# def extract_components(question):
#     input_text = question
#     inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True).to(device)
#     outputs = model.generate(**inputs, max_length=128)
#     decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
#     return decoded_output

In [127]:
# question = "recommend me a laptop asus have rtx 3060, i9 12900H, 16gb ram, 16inch screen"
# result = extract_components(question)
# print(result)